In [1]:
import numpy as np
import shorttext

/Users/stephenhky/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# loading the whole ESV Bible
import biblebooks.bibledocs_iterator as bit
dbconn = bit.get_sqlite3_dbconn("ESVCrossway.sqlite")
biblecorpus = bit.generate_classdict_chapters(dbconn)

In [7]:
# loading FastText model
wvmodel = shorttext.utils.load_fasttext_model('/Users/stephenhky/Data/FastText/wiki.simple/wiki.simple.bin')

In [4]:
# define neural network

In [12]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Activation, Input
from keras.models import Sequential, Model
from keras.regularizers import l2

In [14]:
input_x = Input(shape=(None, wvmodel.vector_size))
output_y = LSTM(1024)(input_x)
output_y = Conv1D(1200, 120)(output_y)
output_y = MaxPooling1D(100)(output_y)
output_y = Dense(2, activation='sigmoid')

kmodel = Model(inputs=input_x, outputs=output_y)

ValueError: Input 0 is incompatible with layer conv1d_2: expected ndim=3, found ndim=2

In [8]:
wvmodel.vector_size

300